In [ ]:
from datetime import datetime
import json
import pandas as pd
import requests

APP_ID = "secret"
BASE_CURRENCY = "EUR"

In [ ]:
from google.cloud import bigquery, storage
bq_client = bigquery.Client()
storage_client = storage.Client()
def csv_to_bq(project_id, dataset_id, destination_table_id, bucket_name, source_file_name, table_schema):
    destination_blob_name = dataset_id + '/' + destination_table_id + '.csv'

    """
    Uploads a file to the bucket.
    https://cloud.google.com/storage/docs/uploading-objects
    """
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('Uploaded file {} to {}.'.format(source_file_name, destination_blob_name))

    """
    Load CSV from GCS to BQ
    https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv
    """
    dataset_ref = bq_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.schema = table_schema
    job_config.skip_leading_rows = 1
    job_config.max_bad_records = 100
    job_config.field_delimiter = ","
    job_config.source_format = bigquery.SourceFormat.CSV # The source format defaults to CSV, so this line is optional.
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE # https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv#loading_csv_data_with_schema_auto-detection
    job_config.allow_quoted_newlines = True

    uri = 'gs://' + bucket_name + '/' + destination_blob_name

    load_job = bq_client.load_table_from_uri(
        uri,
        dataset_ref.table(destination_table_id),
        job_config=job_config)  # API request
    print('Starting job {} from {}'.format(load_job.job_id, uri))

    load_job.result()  # Waits for table load to complete.
    print('Job finished.')

    destination_table = bq_client.get_table(dataset_ref.table(destination_table_id))
    print('Resulted table has {} rows.'.format(destination_table.num_rows))
    
    return load_job

In [ ]:
# r = requests.get("https://openexchangerates.org/api/latest.json", params={'app_id': APP_ID, 'base': BASE_CURRENCY})

r = requests.get("https://openexchangerates.org/api/latest.json", params={'app_id': APP_ID}, verify=False)
data = r.json()
request_time = datetime.now()

In [ ]:
data_list = [[data['base'], key, data['rates'][key]] for key in data['rates']]

df = pd.DataFrame(data_list, columns=["from", "to", "rate"])
df["date"] = request_time.strftime("%Y%m%d")
df["datetime"] = request_time.strftime("%Y%m%d%H%M%S")
df["timestamp"] = int(request_time.timestamp())

tablename = "openexchangerates_" + str(int(request_time.timestamp()))
filename = "./" + tablename + ".csv"

df.to_csv(filename, encoding='utf-8', index=False)

project_id = 'project_id'
dataset_id = 'dataset_id'
destination_table_id = tablename
bucket_name = 'bucket_name'
source_file_name = filename
table_schema = [
    bigquery.SchemaField('from', 'STRING'),
    bigquery.SchemaField('to', 'STRING'),
    bigquery.SchemaField('rate', 'FLOAT'),
    bigquery.SchemaField('date', 'STRING'),
    bigquery.SchemaField('datetime', 'STRING'),
    bigquery.SchemaField('timestamp', 'TIMESTAMP'),
]

csv_to_bq(project_id, dataset_id, destination_table_id, bucket_name, source_file_name, table_schema)